# Hyperparameter Tuning

### Tuning Process

심층 신경망을 훈련시킬 때 설정해야 하는 하이퍼파라미터의 수는 굉장히 많습니다. 그 중에서도 학습률(α)는 가장 중요하게 고려해야 하는 하이퍼파라미터입니다. 다른 하이퍼파라미터 중 몇몇은 α 다음으로 중요할 수 있습니다. 그럼에도 불구하고, 어떤 하이퍼파라미터가 중요한지 미리 알 수 없기 때문에 효과적인 하이퍼파라미터 탐색 방법이 필요합니다.

전통적인 기계 학습 알고리즘에서는 하이퍼파라미터의 값들을 그리드(grid) 형식으로 샘플링하여 시스템적으로 탐색하곤 했습니다. 그러나 깊은 학습에서는 무작위로 샘플링하는 것이 권장됩니다. 이유는 무작위 샘플링을 통해 중요한 하이퍼파라미터의 가능한 값을 더 풍부하게 탐색할 수 있기 때문입니다.

또한, 무작위로 하이퍼파라미터를 샘플링할 때 "대략적에서 세밀한(coarse to fine)" 샘플링 방식을 사용하는 것이 좋습니다. 즉, 처음에는 넓은 범위에서 무작위로 샘플링을 수행한 후, 가장 좋은 결과를 보이는 영역을 좁혀 그 안에서 다시 무작위 샘플링을 수행하는 방식입니다.

결론적으로, 하이퍼파라미터 탐색 과정을 더 체계적으로 구성하려면 무작위 샘플링을 사용하고, 필요한 경우 대략적에서 세밀한 탐색 과정을 구현하는 것이 좋습니다.

### Using an Appropriate Scale to pick Hyperparameters


1. **균일 샘플링 vs 로그 스케일 샘플링**:
    - 하이퍼파라미터의 적절한 값을 찾기 위해 랜덤 샘플링을 사용할 때, 단순히 모든 가능한 값들을 균일하게 샘플링하는 것은 항상 최적이 아닙니다.
    - 예를 들어, 학습률 α의 적절한 범위를 0.0001에서 1로 가정할 때, 이 범위 내에서 균등하게 샘플링하면 0.1과 1 사이의 값들이 대부분 선택됩니다.
    - 하지만 로그 스케일에서 샘플링을 수행하면 각 스케일마다 균등한 확률로 값을 선택하게 되어 더 다양한 학습률 값을 탐색할 수 있습니다.

2. **로그 스케일 샘플링 방법**:
    - 로그 스케일에서 샘플링하려면 첫째, 로그 베이스 10을 사용하여 하이퍼파라미터의 최소값과 최대값의 로그 값을 찾습니다. 
    - 이 범위 내에서 랜덤 값을 선택합니다. 
    - 선택한 랜덤 값을 10의 거듭제곱으로 변환하여 새로운 하이퍼파라미터 값을 생성합니다.

3. **지수 가중 평균의 β 샘플링**:
    - β는 지수 가중 평균을 계산할 때 사용되는 값으로, 그 의미는 '최근 몇 개의 값을 얼마나 중요하게 볼 것인가?'입니다. β 값이 0.9라면 최근 10개의 값, 0.999라면 최근 1,000개의 값을 고려한다는 것을 의미합니다.
    - 0.9에서 0.999 사이에서 β를 균일하게 샘플링하면, 상당수의 값들이 0.99와 0.999 사이에 집중되게 됩니다. 
    - 그러나 여기서는 1 - β 값을 로그 스케일로 샘플링하여 β 값을 얻는 방법을 사용하면, β 값의 탐색 범위가 균등해집니다.

4. **왜 이렇게 샘플링하는가?**:
    - β가 1에 가깝게 될수록 결과는 β의 아주 작은 변화에도 민감하게 반응합니다. 예를 들어, β가 0.9에서 0.9005로 조금 변할 때의 차이는 크지 않지만, 0.999에서 0.9995로 변할 때의 차이는 상당히 큽니다.
    - 이러한 성질 때문에 β가 1에 가까운 구간에서 더 밀집된 샘플링이 필요합니다.

5. **결론**:
    - 하이퍼파라미터 탐색을 할 때는 적절한 스케일에서 샘플링하는 것이 중요합니다. 
    - 하지만, 처음부터 완벽한 스케일에서 샘플링하지 않더라도 여러 차례 탐색을 통해 좋은 결과를 얻을 수 있습니다. 그리고 이후 탐색에서는 더 좋은 결과를 얻기 위해 탐색 범위를 좁혀나갈 수 있습니다.

이 내용은 하이퍼파라미터 탐색을 효과적으로 수행하기 위한 방법 중 하나로, 실제로 모델 학습과 검증을 통해 최적의 값을 찾아가는 과정에서 도움을 줍니다.

## Hyperparameters Tuning in Practice: Pandas vs. Caviar

1. **하이퍼파라미터 탐색의 중요성**:
    - 딥러닝은 다양한 분야에 적용되며, 하나의 응용 분야에서의 하이퍼파라미터 설정에 대한 직관이 다른 분야로 전이되지 않을 수 있습니다.
    - 하지만 서로 다른 응용 분야 간에는 많은 상호 작용이 있습니다. 예로, 컴퓨터 비전 분야에서 개발된 아이디어가 음성 분야로, 음성에서 개발된 아이디어가 자연어 처리 분야로 전이되기도 합니다.
    - 다양한 응용 분야에서 연구를 통해 하이퍼파라미터의 최적값을 찾는 것이 중요하다.

2. **하이퍼파라미터 직관의 소멸**:
    - 하나의 문제에만 초점을 맞추더라도, 알고리즘이나 데이터가 시간이 지나면서 변화할 수 있습니다. 이로 인해 하이퍼파라미터의 최적값이 변할 수 있습니다.
    - 그렇기에 몇 개월마다 하이퍼파라미터를 재평가하는 것이 좋습니다.

3. **하이퍼파라미터 탐색 방식**:
    - 대표적으로 두 가지 접근 방식이 있습니다.
        1. **모델 한 개 돌보기 (Panda 접근법)**: 
            - 이 방법은 대용량의 데이터셋이 있지만, 많은 계산 리소스가 없는 경우에 적합합니다.
            - 모델을 학습하면서 하루하루 성능을 확인하고 하이퍼파라미터를 조절하는 방법입니다.
        2. **여러 모델 병렬 학습 (Caviar 접근법)**:
            - 여러 하이퍼파라미터 설정으로 동시에 다양한 모델을 학습시킵니다.
            - 모든 모델이 학습된 후 가장 좋은 성능을 보이는 모델을 선택합니다.

4. **Panda vs Caviar**:
    - **Panda 접근법**: 팬더는 한 번에 하나의 아기만을 가지며, 그 아기를 꼼꼼히 돌보는 방식입니다. 이와 유사하게 모델 하나에 집중하여 조금씩 조절하며 학습시키는 방법입니다.
    - **Caviar 접근법**: 어떤 물고기는 한 번에 수많은 알을 낳습니다. 그 중에서 살아남는 알만을 선택하는 방식입니다. 이와 유사하게 여러 모델을 동시에 학습시키고, 가장 좋은 성능의 모델만을 선택하는 방법입니다.

딥러닝에서 하이퍼파라미터 탐색은 매우 중요한 과정입니다. 이를 효과적으로 수행하려면 자신의 상황(데이터 크기, 계산 리소스 등)에 따라 적절한 방법을 선택해야 합니다.

# Batch normalization

## Normalizing Activations in a Network

1. **정의**:
   - 배치 정규화는 심층 학습에서 중요한 알고리즘이며, 입력 특징의 평균과 분산을 정규화하는 것처럼, 은닉층의 활성화 값도 정규화하는 과정입니다.

2. **창시자**:
   - Sergey Ioffe와 Christian Szegedy에 의해 개발되었습니다.

3. **목적**:
   - 하이퍼파라미터의 선택 범위를 확장합니다.
   - 심층 신경망의 훈련을 더욱 효율적이고 안정적으로 만듭니다.
   
4. **작동 원리**:
   - **정규화 과정**: 은닉층의 값 `z` (활성화 함수 적용 전의 값)을 정규화하여 평균이 0이고 분산이 1이 되도록 합니다.
   - **조정 파라미터**: `gamma`와 `beta`라는 학습 가능한 파라미터를 도입하여 정규화된 값의 평균과 분산을 조정합니다.

5. **응용 방식**:
   - 정규화는 입력 특징뿐만 아니라 깊은 은닉층의 값들에도 적용됩니다.
   - 은닉층의 활성화 값이 특정 분포를 갖도록 만들고자 할 때, `gamma`와 `beta` 값을 조절하여 원하는 분포를 얻을 수 있습니다.

6. **결과 및 효과**:
   - 학습 속도의 향상과 함께 심층 신경망의 안정성이 증가합니다.
   - 하이퍼파라미터 선택의 유연성이 증가하여 다양한 설정에서도 모델이 잘 동작하게 됩니다.
   

## Fitting Batch Norm into a Neural Network

깊은 신경망에서 각 층에서 두 단계의 연산이 있습니다. Z 값을 계산하고 활성화 함수를 적용하여 A 값을 얻습니다. 만약 Batch Norm을 적용하지 않으면 입력 X는 첫 번째 은닉층에 들어가 Z1을 계산합니다. 그리고 활성화 함수를 통해 A1 값을 얻습니다. 하지만 Batch Norm을 적용할 경우 Z1 값에 Batch Norm을 적용하고, 그 결과로 정규화된 값 Z tilde 1을 얻습니다. 그리고 Z tilde 1을 활성화 함수에 입력하여 A1 값을 얻습니다.

이와 같은 방식으로 신경망의 모든 층에서 Batch Norm을 적용할 수 있습니다. 핵심은 Z 값을 직접 사용하는 대신 정규화된 값을 사용하는 것입니다.

이제 Batch Norm을 적용하면 신경망의 파라미터는 W와 B 뿐만 아니라 Beta와 Gamma도 포함됩니다. 그런데 Batch Norm에서는 Z의 평균을 0으로 만들기 때문에 B 파라미터는 필요 없게 됩니다. 그 결과 실제로 사용되는 파라미터는 W, Beta, Gamma만이 됩니다.

Batch Norm의 또 다른 중요한 점은 전체 학습 데이터셋을 한 번에 사용하는 것이 아니라 미니 배치 단위로 적용된다는 것입니다. 즉, 각 미니 배치마다 Z 값을 정규화합니다.

마지막으로, Batch Norm을 적용한 신경망을 학습할 때는 일반적인 경사하강법 뿐만 아니라 모멘텀, RMSprop, Adam과 같은 최적화 알고리즘을 함께 사용할 수 있습니다.

요약하면, Batch Norm은 심층 신경망 학습 속도를 향상시키기 위해 각 층의 출력 값을 정규화하는 방법입니다. 이로 인해 학습이 빠르고 안정적으로 진행됩니다. 프로그래밍 프레임워크를 사용하면 Batch Norm을 쉽게 구현할 수 있습니다.

## Why does Batch Norm work?

#### 1. 배치 정규화의 기본 아이디어

- 입력 특징들을 정규화하면 학습이 더 빨라집니다. 이 아이디어를 신경망의 모든 층에 적용한 것이 배치 정규화입니다.
- 즉, 각 층의 입력을 평균 0과 분산 1로 정규화하면 학습이 더 잘 되는 경향이 있습니다.

#### 2. Covariate Shift

- 'Covariate Shift'는 학습 데이터의 분포가 시간에 따라 변하는 현상을 의미합니다.
- 예를 들어, 검은색 고양이만으로 학습한 모델이 다양한 색상의 고양이를 잘 인식하지 못하는 문제가 있을 수 있습니다.
- 심층 신경망에서도 이 문제가 발생할 수 있는데, 예를 들어 첫 번째 층의 가중치가 바뀌면 그 다음 층의 입력 분포도 바뀌게 됩니다.

#### 3. 심층 신경망에서의 Covariate Shift

- 신경망의 앞쪽 층이 바뀌면 뒤쪽 층의 입력 분포도 변화합니다.
- 배치 정규화를 사용하면 이러한 변화를 최소화할 수 있습니다. 이는 각 층이 더 안정적으로 학습될 수 있게 해 줍니다.
- 결과적으로, 앞쪽 층의 작은 변경이 뒤쪽 층에 큰 영향을 미치는 것을 방지하게 됩니다.

#### 4. 일반화 및 정규화 효과

- 배치 정규화는 미니 배치의 평균 및 분산에 기반한 정규화를 진행하므로, 각 미니 배치마다 약간의 노이즈가 발생합니다.
- 이 노이즈는 약간의 정규화 효과를 제공합니다. 이는 신경망이 특정 은닉 유닛에 과도하게 의존하지 않게 만들어줍니다.
- 하지만 이러한 정규화 효과는 크지 않으므로 필요에 따라 다른 정규화 기법과 함께 사용될 수 있습니다.

#### 5. 테스트 시의 배치 정규화

- 학습 시에는 미니 배치의 평균 및 분산을 사용합니다.
- 그러나 실제 예측(테스트)을 할 때는 전체 학습 데이터셋의 평균과 분산을 사용하여 정규화해야 합니다.

배치 정규화는 학습 도중 각 층의 입력 분포를 안정화시켜 학습 속도를 향상시키는 기법입니다. 덕분에 더 빠르게 학습하면서도 좀 더 안정적인 모델을 얻을 수 있습니다.

## Batch Norm at Test Time

### 배치 정규화의 학습 및 테스트 과정

#### 1. **학습 시점의 배치 정규화**:
- 학습 동안, 데이터는 미니 배치 단위로 처리됩니다. 이때 각 미니 배치의 평균(μ)와 분산(σ^2)을 계산합니다.
- 평균은 미니 배치 내의 모든 예제에 대해 Z값들을 합산함으로써 계산됩니다.
- 이후 Z값을 정규화하여 Znorm을 얻습니다: $( Z_{norm} = \frac{Z - μ}{\sqrt{σ^2 + ε}} $). 여기서 ε은 수치 안정성을 위해 추가됩니다.
- 마지막으로, Znorm에 스케일 및 시프트를 적용하여 최종 값을 얻습니다: $( \tilde{Z} = γ × Z_{norm} + β $). 여기서 γ와 β는 학습 가능한 파라미터입니다.

#### 2. **테스트 시점의 배치 정규화**:
- 테스트 시에는 보통 한 번에 하나의 예제만 처리할 수 있습니다. 따라서 μ와 σ^2을 직접 계산하는 것이 무의미합니다.
- 이 문제를 해결하기 위해, 학습 도중 각 미니 배치에서 관측된 μ와 σ^2의 지수 가중 평균 (exponentially weighted average)을 사용하여 μ와 σ^2을 추정합니다.
- 즉, 학습 과정에서 각 미니 배치마다 계산된 μ와 σ^2를 추적하고, 이를 통해 전체 데이터셋에 대한 평균 μ와 σ^2의 추정치를 얻습니다.
- 테스트 시에는 이 추정치를 사용하여 Z값을 정규화하고, 학습된 γ와 β 파라미터를 사용하여 최종 출력 값을 계산합니다.

#### 3. **요약**:
- 학습 시간 동안, μ와 σ^2는 미니 배치 전체에서 계산됩니다.
- 그러나 테스트 시에는 전체 학습 데이터셋에서 추정된 μ와 σ^2 값을 사용하여 예측을 수행합니다.
- 이러한 접근 방식은 신경망이 학습 데이터의 실제 분포를 잘 반영하도록 도와주며, 실제 세계의 다양한 데이터에 잘 일반화될 수 있게 합니다.

---

이처럼 배치 정규화는 학습과 테스트 과정에서 약간 다르게 적용됩니다. 그럼에도 불구하고 이 기술은 딥러닝 모델의 학습 속도를 크게 향상시키고, 더 깊은 네트워크를 효과적으로 학습시키는 데 매우 중요합니다.